In [11]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
from numba import jit
import pickle

from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args

Data provided must be in the following form:

datesA closeA datesB closeB datesC closeC ...

This method prevents survivorship bias if one just selected current members of S&P 500 index (or any other index).

In [2]:
close = pd.read_parquet('spx_close.parquet')

In [3]:
close.head()

,date,lyb,date.1,axp,date.2,vz,date.3,avgo,date.4,ba,...,date.1008,667517q,date.1009,300583q,date.1010,285939q,date.1011,1727044d,date.1012,827663q
0,2013-01-02,59.17,2000-01-03,45.8828,2000-01-03,53.7247,2015-01-02,100.09,2000-01-03,40.188,...,2000-01-03,59.6250,2000-01-03,84.1250,2000-01-03,44.0625,2000-01-03,63.8125,2000-01-03,86.0000
1,2013-01-03,57.64,2000-01-04,44.1504,2000-01-04,51.9880,2015-01-05,98.49,2000-01-04,40.125,...,2000-01-04,56.6250,2000-01-04,80.8750,2000-01-04,43.5000,2000-01-04,64.6875,2000-01-04,79.6250
2,2013-01-04,58.41,2000-01-05,42.9650,2000-01-05,53.7247,2015-01-06,96.25,2000-01-05,42.625,...,2000-01-05,54.1250,2000-01-05,79.9375,2000-01-05,44.1875,2000-01-05,65.5000,2000-01-05,77.1875
3,2013-01-07,59.14,2000-01-06,43.8404,2000-01-06,53.1085,2015-01-07,98.85,2000-01-06,43.063,...,2000-01-06,53.9375,2000-01-06,80.6250,2000-01-06,47.2500,2000-01-06,65.7500,NaT,NaN
4,2013-01-08,59.74,2000-01-07,44.4786,2000-01-07,52.7163,2015-01-08,103.79,2000-01-07,44.313,...,2000-01-07,58.0000,2000-01-07,80.4375,2000-01-07,49.9375,2000-01-07,65.7500,NaT,NaN


In [4]:
def get_tick_df(x, close, split=pd.to_datetime('2017-01-01')):
    """
    Identifies and splits data into 'train' and 'test' parts
    
    split - date to split into 'test' and 'train' parts
    """
    
    tick_df = close.iloc[:,x:x+2]
    tick = tick_df.columns[1]
    tick_df.columns = ['date', tick]
    tick_df = tick_df.set_index('date').dropna()
    train = tick_df.loc[:split]
    test = tick_df.loc[split-pd.Timedelta(100, 'D'):]
    
    return train, test

In [5]:
x = 0

train_df = {}
test_df = {}
split = pd.to_datetime('2019-03-01')

for _ in tqdm_notebook(range(int(close.shape[1]/2))):
    
    tick = close.iloc[:,x+1].name
    if close[tick].dropna().shape[0] == 0:
        # some are empty, these are ignored
        x+=2
        continue
    else:
        train_df[tick], test_df[tick] = get_tick_df(x, close=close, split=split)
        x+=2


In [6]:
@jit(nopython=True)
def ema_rsi(series, a):
    """
    numba-accelerated loop to calculate EMA/Wilder rsi
    series - np.array of values, not pd.Series!
    a - alpha, or decay parameter of RSI calculation
    """
    prev = series[0]
    ema_series = [prev]
    
    for v in series[1:]:
        new_val = a*v + (1-a)*prev
        ema_series.append( new_val )
        prev = new_val
    
    return ema_series

def rsi(frame, days=14, method='ws'):
    """
    calculates rsi series based on provided parameters
    
    days - period to calculate the RSI
    method - 'ws' (Wilder'), 'sma' (Simple Moving Average), 'ema' (Exponential Moving Average)
    """
    tick = frame.columns[0]
    frame['change'] = frame[tick] - frame[tick].shift(1)
    frame['up_move'] = np.where( frame['change'] > 0, frame['change'], 0 )
    frame['down_move'] = np.abs(np.where( frame['change'] < 0, frame['change'], 0 ))
    
    if method == 'sma':
        frame['avg_up'] = frame['up_move'].rolling(days).mean()
        frame['avg_down'] = frame['down_move'].rolling(days).mean()
        
    elif method == 'ema':
        alpha = 2/(days+1)
        frame['avg_up'] = ema_rsi( frame['up_move'].values, alpha )
        frame['avg_down'] = ema_rsi( frame['down_move'].values, alpha )
    
    else:
        alpha = 1/days
        frame['avg_up'] = ema_rsi( frame['up_move'].values, alpha )
        frame['avg_down'] = ema_rsi( frame['down_move'].values, alpha )
    
    frame['rs'] = frame['avg_up']/frame['avg_down']
    frame['rsi'] = 100 - 100/(1+frame['rs'])
    frame['rsi'].iloc[:days] = np.nan
    
    return frame['rsi']

In [7]:
class Stock_history():
    
    def __init__(self, df, rsi_days=14, calc_type='ws'):
        """
        Calculates rsi for this particular stock
        """
        self.df = df
        self.tick = df.columns[0]
        self.df['rsi'] = rsi(self.df[[self.tick]], days=rsi_days, method=calc_type)

    def get_signals(self, **key_args):
        """
        calculates signals for the stock with given keywords
        and then calculates geometric mean of trades' return 
        
        fwd - holding period after purchasing a stock
        low - minimum period after previous low to find a new low in days
        high - maximum period after previous low to find a new low in days
        step - how often to look for lower low between 'low' and 'high'
        rsi1 - maximum RSI value during the first low
        rsi2 - maximum RSI value during the second low
        rsi_chng - minimum rsi increase from first to second low
                   Attempts to find an increase in RSI while the price goes further down
        price_chng - maximum price change from first to second low
                     Attempts to find an increase in price while the RSI goes up
        threshold - minimum number of signals required to consider this a True buy recommendation
                    Maximum number of signals is int((high-low)/step), so there can be more than 1 signal in a single day.
                    This threshold may decrease the algorithm's false positives. 
        """
        
        #### uses default kwargs if not all are provided
        kwargs = {
            'fwd': 5,
            'low': 5,
            'high': 21,
            'step': 5,
            'rsi1': 30,
            'rsi2': 35,
            'rsi_chng': 0,
            'price_chng': -0.01,
            'threshold': 2
        }
        
        for k, v in key_args.items():
            if k in kwargs.keys():
                kwargs[k] = v
                

        self.df['signal'] = 0
        self.df['fwd'] = self.df[self.tick].shift(-kwargs['fwd'])/self.df[self.tick]-1

        for x in range(kwargs['low'], kwargs['high'], kwargs['step']):
            
            self.df['d'+str(x)] = self.df[self.tick]/self.df[self.tick].shift(x)-1
            self.df['r'+str(x)] = self.df['rsi']-self.df['rsi'].shift(x)
            self.df['r_'+str(x)] = self.df['rsi'].shift(x)
            
            self.df['signal'] = np.where( (self.df.rsi <= kwargs['rsi2']) & 
                                         (self.df['r_'+str(x)] <= kwargs['rsi1']) & 
                                           (self.df['r'+str(x)] > kwargs['rsi_chng']) & 
                                         (self.df['d'+str(x)] < kwargs['price_chng']),
                                   self.df['signal'] + 1, self.df['signal'])
        
        self.geoslice = self.df[self.df.signal >= kwargs['threshold']][[self.tick, 'rsi', 'signal', 'fwd']]
        self.geoslice.columns = ['tick', 'rsi', 'signal', 'fwd']
        self.geoslice['tick'] = self.tick
        self.geomean = np.prod(self.df[self.df.signal >= kwargs['threshold']]['fwd']+1)-1
        self.count = self.df[self.df.signal >= kwargs['threshold']].shape[0]

In [8]:
#### Checking if stuff works:

ticker = 'aa'
itick = Stock_history(train_df[ticker])
itick.get_signals()
print(itick.geomean, itick.count)
itick.geoslice

0.04731301257959908 37


,tick,rsi,signal,fwd
date,,,,
2000-02-07,aa,29.944085,2,0.060551
2002-06-24,aa,30.287920,2,0.071428
2002-07-15,aa,30.815151,2,-0.148973
2005-10-11,aa,24.744055,2,0.018818
2005-10-12,aa,23.878613,2,0.044473
2005-10-14,aa,31.049742,2,0.016543
2008-10-13,aa,25.808165,2,-0.102024
2008-10-16,aa,26.224120,2,-0.181818
2008-10-20,aa,28.471893,3,-0.271555


The following is a Bayesian optimizatio of parameters listed in 'get_signals' method of Stock_history class. 
It attempts to find the best parameters withing some bounds.

In [10]:
search_space = [
    
    Integer(2, 15, name='fwd'),
    Integer(3, 12, name='low'),
    Integer(12, 50, name='high'),
    Integer(1, 4, name='step'),  
    Integer(30, 40, name='rsi1'),
    Integer(20, 30, name='rsi2'),   
    Real(0, 2, prior='uniform', name='rsi_chng'),    
    Real(-0.05, 0, prior='uniform', name='price_chng'),   
    Integer(1, 4, name='threshold'),
]

default_params = {
    'fwd': 5,
    'low': 5,
    'high': 21,
    'step': 5,
    'rsi1': 30,
    'rsi2': 35,
    'rsi_chng': 0,
    'price_chng': -0.01,
    'threshold': 2
}

@use_named_args(search_space)
def assess_kwargs(**kwargs):
    """
    Assesses parameters and returns a value based on the geometric mean of returns divided
    by the standard deviation of returns to penalize for volatility.
    
    Low number of predictions (i.e. less than 500) can be additionally penalized. 
    """
    print(kwargs)
    stocks = {}
    slices = {}

    for k, v in tqdm_notebook(train_df.items()):

        stocks[k] = Stock_history(v)
        stocks[k].get_signals(**kwargs)
        slices[k] = stocks[k].geoslice

    geoslice = pd.concat(slices.values()).sort_index()
    geoslice['date_'] = geoslice.index
    geoslice = geoslice.sort_values(by=['date', 'signal']).drop_duplicates(subset=['date_'])
    
    #Full penalty for extremely low number of predictions
    if geoslice.shape[0] < 50:
        return 0
    #Proportionally penalizes for low number of predictions
    elif geoslice.shape[0] < 500:
        return -geoslice['fwd'].mean()/geoslice['fwd'].std() * geoslice.shape[0]/500
    else:
        return -geoslice['fwd'].mean()/geoslice['fwd'].std()

In [ ]:
"""
Bayesian optimization process to find optimal parameters.

Saves them to disk afterwards for future use.
"""

result = gp_minimize(assess_kwargs, search_space, random_state=17, n_calls=50, verbose=True, n_initial_points=20)
opt_params = dict(zip(default_params.keys(), result.x))

with open('mid_rsi_params.dict', 'wb') as config_dictionary_file:
 
    pickle.dump(opt_params, config_dictionary_file)

In [ ]:
"""
Testing parameters on test data. 
"""

stocks = {}
slices = {}

for k, v in tqdm_notebook(test_df.items()):

    stocks[k] = Stock_history(v)
    stocks[k].get_signals(**opt_params)
    slices[k] = stocks[k].geoslice

geoslice = pd.concat(slices.values()).sort_index()
geoslice['date_'] = geoslice.index
geoslice = geoslice.sort_values(by=['date', 'signal']).drop_duplicates(subset=['date_'])

print(geoslice['fwd'].mean(), geoslice['fwd'].median(), geoslice['fwd'].std())